In [1]:
import torch
from selfcheckgpt.modeling_mqag import MQAG

/home/cwtang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(28)

In [14]:
!export CUDA_VISIBLE_DEVICES=2

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [21]:
mqag_model = MQAG(
    g1_model_type='race', # race (more abstractive), squad (more extractive)
    device='cpu'
    # device=device
)

MQAG (race) initialized to cpu


# MQAG Score: `score`
- Using MQAG for assessing the consistency between document and summary as described in: https://arxiv.org/abs/2301.12307

In [22]:
document = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

In [23]:
summary = "Djokvic might be allowed to play in the US next month. Djokovic will play in Qatar next week after winning his 5th Grand slam."

In [24]:
score = mqag_model.score(candidate=summary, reference=document, num_questions=3, verbose=True)

Initialized Generation
Initialized Answering
Q1: Djokovic is allowed to play in _ next month.
(1) [P(.|cand)=89.17%]	[P(.|ref)=95.65%]	the US
(2) [P(.|cand)=0.99%]	[P(.|ref)=1.13%]	Japan
(3) [P(.|cand)=1.24%]	[P(.|ref)=1.29%]	Germany
(4) [P(.|cand)=8.61%]	[P(.|ref)=1.92%]	Qatar
-------------------------------------------------------------------------------
Q2: Djokovic won't play in _ USA next month.
(1) [P(.|cand)=7.28%]	[P(.|ref)=7.84%]	London
(2) [P(.|cand)=10.68%]	[P(.|ref)=30.13%]	Paris
(3) [P(.|cand)=6.11%]	[P(.|ref)=11.64%]	Sydney
(4) [P(.|cand)=75.93%]	[P(.|ref)=50.40%]	New York
-------------------------------------------------------------------------------
Q3: Djokovic will play in _.
(1) [P(.|cand)=99.96%]	[P(.|ref)=0.67%]	Qatar
(2) [P(.|cand)=0.01%]	[P(.|ref)=0.44%]	China
(3) [P(.|cand)=0.02%]	[P(.|ref)=0.74%]	the UK
(4) [P(.|cand)=0.02%]	[P(.|ref)=98.15%]	the US
-------------------------------------------------------------------------------


In [25]:
print(score)

{'kl_div': 1.740960064186744, 'counting': 0.3333333333333333, 'hellinger': 0.4515573396813452, 'total_variation': 0.43834160392483074}


In [26]:
print("KL-div    =", score['kl_div'])
print("Counting  =", score['counting'])
print("Hellinger =", score['hellinger'])
print("Total Var =", score['total_variation'])

KL-div    = 1.740960064186744
Counting  = 0.3333333333333333
Hellinger = 0.4515573396813452
Total Var = 0.43834160392483074


# Multiple-choice Question Generation: `generate`

In [9]:
context = r"""
World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
to play at Indian Wells and the Miami Open next month. The United States has extended
its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
will be required to enter the country until at least 10 April, but the Serbian has previously
said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")

In [10]:
questions = mqag_model.generate(context=context, do_sample=True, num_questions=3)
for i, question_item in enumerate(questions):
    print("------------------------------------")
    print(f"Q{i}: {question_item['question']}")
    print(f"A: {question_item['options'][0]}")
    print(f"B: {question_item['options'][1]}")
    print(f"C: {question_item['options'][2]}")
    print(f"D: {question_item['options'][3]}")

------------------------------------
Q0: How many Grand Slam men's titles has Djokovic won?
A: 22
B: 10
C: 20
D: 24
------------------------------------
Q1: Why is Djokovic hoping for a "positive decision"?
A: To be allowed to play at Indian Wellsand the Miami Open.
B: To play at the Miami Open.
C: To apply to stay at the US until at least 10 April.
D: To have his proof of vaccination.
------------------------------------
Q2: What does the author want to tell readers?
A: Djokovic won't leave the ATP tour in Dubai.
B: Djokovic will play at Indian Wells and the Miami Open next month.
C: Djokovic won't enter the country until at least 10 April.
D: The United States doesn't need foreign visitors to be vaccinated again.


# Multiple-choice Question Answering: `answer`

In [11]:
context = r"""Chelsea's mini-revival continued with a third victory in a row as they consigned struggling Leicester City to a fifth consecutive defeat.
Buoyed by their Champions League win over Borussia Dortmund, Chelsea started brightly and Ben Chilwell volleyed in from a tight angle against his old club.
Chelsea's Joao Felix and Leicester's Kiernan Dewsbury-Hall hit the woodwork in the space of two minutes, then Felix had a goal ruled out by the video assistant referee for offside.
Patson Daka rifled home an excellent equaliser after Ricardo Pereira won the ball off the dawdling Felix outside the box.
But Kai Havertz pounced six minutes into first-half injury time with an excellent dinked finish from Enzo Fernandez's clever aerial ball.
Mykhailo Mudryk thought he had his first goal for the Blues after the break but his effort was disallowed for offside.
Mateo Kovacic sealed the win as he volleyed in from Mudryk's header.
The sliding Foxes, who ended with 10 men following Wout Faes' late dismissal for a second booking, now just sit one point outside the relegation zone.
""".replace('\n', ' ')

question = "Who had a goal ruled out for offside?"
options  = ['Ricardo Pereira', 'Ben Chilwell', 'Joao Felix', 'The Foxes']

In [12]:
questions = [{'question': question, 'options': options}]
probs = mqag_model.answer(questions=questions, context=context)
print(probs[0])

[0.00145158 0.00460854 0.99049687 0.00344299]
